In [ ]:
import sys
import tensorflow as tf
sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/")

# import customized modules
import preprocess, data_generator, check_result, metrics, loss, train

sys.path.append("/visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet")
import dense_unet

# Insert Path
# {model_name}_{epoch}ep_{model_inpusize}_{Generator | aG/fG/sG}
FILE_NAME = 'dense_unet_40ep_512_rcG_wo_decay'
INPUT_SHAPE = (256, 256, 1)
MODEL_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/" + FILE_NAME + ".tf"
HISTORY_PATH = "/visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/history/" + FILE_NAME + '_history' ".json"
TSOURCE = '/visuworks/Dataset/Selected Dataset 3/Train'

TRAIN_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Train'
TEST_SOURCE = '/visuworks/Dataset/Selected Dataset 3/Test'

2024-01-30 16:52:52.543886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 16:52:52.543913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 16:52:52.545000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 16:52:52.550500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 16:52:53.128536: W tensorflow/compiler/tf2

In [ ]:
# Define custom objects for loading the model
custom_objects = {'DiceLoss': loss.DiceLoss(), 
                  'sensitivity': metrics.sensitivity,
                  'specificity': metrics.specificity,
                  'accuracy' : metrics.accuracy}


# Create augmentation
train_preproc = preprocess.build_augmentation_for_adaptive()
test_preproc = preprocess.build_augmentation_for_adaptive(is_train=False)

# Create train, test generator
train_generator = data_generator.RandomlyCropGeneraor(
    TRAIN_SOURCE, 
    number_of_images=1,
    input_size=INPUT_SHAPE,
    is_train=True,
    augmentation= train_preproc
)

test_generator = data_generator.RandomlyCropGeneraor(
    TRAIN_SOURCE, 
    number_of_images=1,
    input_size=INPUT_SHAPE,
    is_train=False,
    augmentation= test_preproc
)

# Create model
model = dense_unet.build_model(input_shape=INPUT_SHAPE)
model.summary()

Model: "model"
__________________________________________________________________________________________________


2024-01-30 16:52:54.312844: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 16:52:54.349375: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 16:52:54.349585: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 48)         480       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 48)         192       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 48)         0         ['batch_normalization[0][0]'] 
                                                                                                  
 conv2d_1 

In [3]:
train.model_train(model,
                 40,
                 train_generator,
                 test_generator, 
                 MODEL_PATH,
                 HISTORY_PATH,
                 save_every_epoch = True,
                 apply_weight_decay = False)

Epoch 1/40


2024-01-30 16:52:57.121684: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-30 16:52:58.347657: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f79a267eaf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-30 16:52:58.347679: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-01-30 16:52:58.351059: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706633578.417863  488859 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


880/880 [==============================] - ETA: 0s - loss: 0.4547 - sensitivity: 0.5616 - specificity: 0.9545 - accuracy: 0.9276INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/1--0.4547-0.5616-0.9545-0.5189-0.9678.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/1--0.4547-0.5616-0.9545-0.5189-0.9678.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/1--0.4547-0.5616-0.9545-0.5189-0.9678.pb
880/880 [==============================] - 206s 225ms/step - loss: 0.4547 - sensitivity: 0.5616 - specificity: 0.9545 - accuracy: 0.9276 - val_loss: 0.4420 - val_sensitivity: 0.5189 - val_specificity: 0.9678 - val_accuracy: 0.9433
Epoch 2/40
880/880 [==============================] - ETA: 0s - loss: 0.3789 - sensitivity: 0.6170 - specificity: 0.9705 - accuracy: 0.9471INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/2--0.3789-0.6170-0.9705-0.5016-0.9756.pb/assets


...

INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/4--0.3354-0.6484-0.9754-0.5853-0.9685.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/4--0.3354-0.6484-0.9754-0.5853-0.9685.pb
880/880 [==============================] - 196s 223ms/step - loss: 0.3354 - sensitivity: 0.6484 - specificity: 0.9754 - accuracy: 0.9525 - val_loss: 0.3933 - val_sensitivity: 0.5853 - val_specificity: 0.9685 - val_accuracy: 0.9464
Epoch 5/40
879/880 [============================>.] - ETA: 0s - loss: 0.3402 - sensitivity: 0.6463 - specificity: 0.9753 - accuracy: 0.9532INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/5--0.3402-0.6461-0.9753-0.4869-0.9845.pb/assets


INFO:tensorflow:Assets written to: /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/5--0.3402-0.6461-0.9753-0.4869-0.9845.pb/assets


Model saved to /visuworks/Blindless_AIFFELTON/Ablation Study/Dense_Unet/model_parameters/dense_unet_40ep_512_rcG_wo_decay.tf/5--0.3402-0.6461-0.9753-0.4869-0.9845.pb
880/880 [==============================] - 196s 223ms/step - loss: 0.3402 - sensitivity: 0.6461 - specificity: 0.9753 - accuracy: 0.9531 - val_loss: 0.4071 - val_sensitivity: 0.4869 - val_specificity: 0.9845 - val_accuracy: 0.9533
Epoch 6/40
728/880 [=======================>......] - ETA: 22s - loss: 0.3413 - sensitivity: 0.6360 - specificity: 0.9765 - accuracy: 0.9541

KeyboardInterrupt: 